In [ ]:
! pip install yfinance

  Created wheel for multitasking: filename=multitasking-0.0.10-py3-none-any.whl size=8488 sha256=344a23887bdc7f45b74f443aff98be00f844e56d923bee8f9cca3413871c1c09
  Stored in directory: /root/.cache/pip/wheels/34/ba/79/c0260c6f1a03f420ec7673eff9981778f293b9107974679e36
Successfully built multitasking
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [ ]:
import yfinance as yf
data = yf.download("AAPL IBM", start="2009-01-01", end="2019-12-31")

[*********************100%***********************]  2 of 2 completed


In [ ]:
data['Open']

,AAPL,IBM
Date,,
2009-01-02,12.268571,83.889999
2009-01-05,13.310000,86.419998
2009-01-06,13.707142,87.110001
2009-01-07,13.115714,87.830002
2009-01-08,12.918571,87.809998
...,...,...
2019-12-23,280.529999,135.779999
2019-12-24,284.690002,135.610001
2019-12-26,284.820007,134.979996


In [ ]:
%%bash
$HADOOP_HOME/sbin/start-dfs.sh
$HADOOP_HOME/sbin/start-yarn.sh
$HADOOP_HOME/bin/mapred --daemon start historyserver

Starting namenodes on [localhost]
Starting datanodes
Starting secondary namenodes [p-26eb31ce-99fb-4968-850c-9426b88a1f57]
Starting resourcemanager
Starting nodemanagers


In [ ]:
from hdfs3 import HDFileSystem
hdfs = HDFileSystem(host='localhost', port=9000)
with hdfs.open('AAPL_IBM_open.csv', 'wb') as f:
    data['Open'].to_csv(f,header=True)

In [ ]:
hdfs.ls('.')

['/user/root/AAPL_IBM_open.csv']

In [ ]:
%%file stock_analysis_spark.py
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

spark = SparkSession.builder.appName("Stock analysis").getOrCreate()

mySchema = StructType([
    StructField("date", StringType(), False),
    StructField("apple_opening", FloatType(), False),
    StructField("samsung_opening", FloatType(), False),
    ])

df = spark.read.format("csv").option("header","true").schema(mySchema).load("AAPL_IBM_open.csv")

result = df.select(expr("substring(date,1,4) as year"), expr("substring(date,6,2) as month"), col("apple_opening"), col("samsung_opening")) \
    .filter(col("month").isin(["10","11","12"])) \
    .groupBy("year").agg(max("apple_opening"), max("samsung_opening"))

result.write.format("csv").option("path", "output").save()

Overwriting stock_analysis_spark.py


In [ ]:
! $SPARK_HOME/bin/spark-submit --master yarn stock_analysis_spark.py

2021-12-10 19:25:55,785 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2021-12-10 19:25:57,846 INFO spark.SparkContext: Running Spark version 3.0.3
2021-12-10 19:25:57,910 INFO resource.ResourceUtils: ==============================================================
2021-12-10 19:25:57,911 INFO resource.ResourceUtils: Resources for spark.driver:

2021-12-10 19:25:57,912 INFO resource.ResourceUtils: ==============================================================
2021-12-10 19:25:57,912 INFO spark.SparkContext: Submitted application: Stock Analysis
2021-12-10 19:25:57,996 INFO spark.SecurityManager: Changing view acls to: root
2021-12-10 19:25:57,997 INFO spark.SecurityManager: Changing modify acls to: root
2021-12-10 19:25:57,997 INFO spark.SecurityManager: Changing view acls groups to: 
2021-12-10 19:25:57,997 INFO spark.SecurityManager: Changing modify acls groups to: 
2021-12-10 19:25:57,997 INFO spark.Sec

In [ ]:
! $HADOOP_HOME/bin/hdfs dfs -get output .

get: `output': No such file or directory


In [ ]:
%%bash
$HADOOP_HOME/bin/mapred --daemon stop historyserver
$HADOOP_HOME/sbin/stop-yarn.sh
$HADOOP_HOME/sbin/stop-dfs.sh


Stopping nodemanagers
localhost: WARNING: nodemanager did not stop gracefully after 5 seconds: Trying to kill with kill -9
localhost: ERROR: Unable to kill 845
Stopping resourcemanager
Stopping namenodes on [localhost]
localhost: WARNING: namenode did not stop gracefully after 5 seconds: Trying to kill with kill -9
localhost: ERROR: Unable to kill 253
Stopping datanodes
localhost: WARNING: datanode did not stop gracefully after 5 seconds: Trying to kill with kill -9
localhost: ERROR: Unable to kill 347
Stopping secondary namenodes [p-26eb31ce-99fb-4968-850c-9426b88a1f57]
p-26eb31ce-99fb-4968-850c-9426b88a1f57: WARNING: secondarynamenode did not stop gracefully after 5 seconds: Trying to kill with kill -9
p-26eb31ce-99fb-4968-850c-9426b88a1f57: ERROR: Unable to kill 511
ERROR: Unable to kill 1010
ERROR: Unable to kill 758


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=26eb31ce-99fb-4968-850c-9426b88a1f57' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>